In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/dataset/Anju/fall/data0.csv").drop('timestamp', axis=1)

df.head()

,ax,ay,az,gx,gy,gz
0,-76.94,-0.24,-55.07,868,-11876,7884
1,-21.74,-53.05,-23.84,-12730,-7288,7902
2,-21.46,-52.71,-24.14,-11992,-7628,8076
3,-21.11,-52.31,-24.50,-11186,-7884,7990
4,-20.17,-51.15,-25.53,-9962,-8348,7280


In [ ]:
for col in df.columns:
  print(col,min(df[col]), max(df[col]))

ax -168.99 179.39
ay -53.05 82.23
az -76.49 -3.88
gx -25064 32767
gy -11876 5016
gz -10162 24132


In [ ]:
import os
import random
root = "/content/drive/MyDrive/dataset"
gen = os.walk(root)
list_of_folders = next(gen)[1]
list_of_folders.sort()

train = []
valid = []

for folder in list_of_folders:
  r1 = os.path.join(root, folder)
  g = os.walk(r1)
  
  for subfolder in next(g)[1]:
    r2 = (os.path.join(r1, subfolder))

    files = [f for f in os.listdir(r2) if os.path.isfile(os.path.join(r2, f))]

    for file_name in files:
      if random.uniform(0, 1) < 0.8:
        train.append([os.path.join(r2, file_name), 1 if subfolder == "fall" else 0])
      else:
        valid.append([os.path.join(r2, file_name), 1 if subfolder == "fall" else 0])

# 80 - 20 split to traning & validation data

In [ ]:
import pandas as pd

train_df = pd.DataFrame(train, columns=['path', 'fall'])
valid_df = pd.DataFrame(valid, columns=['path', 'fall'])

In [ ]:
train_df.fall.value_counts()

1    51
0    46
Name: fall, dtype: int64

In [ ]:
shapes = []
import torch

for i in range(len(train_df)):
  csv_path = train_df.iloc[i, 0]
  mat = pd.read_csv(csv_path)
  mat = [list(mat[x]) for x in ['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
  mat = torch.tensor(mat)
  if mat.shape[1] < 10:
    print(csv_path)
  shapes.append(mat.shape[1])

print(shapes)
mat_min = min(shapes)

[126, 121, 120, 124, 121, 120, 123, 131, 108, 127, 127, 129, 134, 127, 132, 138, 134, 132, 138, 139, 133, 136, 136, 133, 140, 137, 136, 134, 136, 135, 136, 135, 128, 129, 141, 136, 132, 92, 137, 141, 142, 138, 137, 144, 139, 142, 121, 123, 120, 128, 122, 119, 128, 130, 126, 128, 128, 133, 131, 88, 130, 132, 134, 133, 135, 142, 134, 137, 134, 129, 133, 132, 134, 134, 135, 132, 135, 133, 134, 136, 127, 127, 129, 128, 129, 129, 131, 132, 128, 136, 137, 134, 138, 136, 137, 138, 136]


In [ ]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, IterableDataset

def mat_to_tensor(mat_path):
  mat = pd.read_csv(mat_path)
  mat = mat.dropna()
  # print(mat.info())
  mat = [list(mat[x]) for x in ['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
  mat = torch.tensor(mat)
  return mat

class IMUSet(Dataset):
    def __init__(self, data_df):
        self.data_df = data_df # data_df == train_df / valid_df

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        mat_path = self.data_df.iloc[idx, 0]
        # print(mat_path)
        t = mat_to_tensor(mat_path)
        label = self.data_df.iloc[idx, 1]
        return t, label

In [ ]:
from torch.utils.data import DataLoader

imuset = IMUSet(train_df) # make dataset

train_dataloader = DataLoader(imuset, shuffle=True) # load data multiple 

In [ ]:
valid_dataset = IMUSet(valid_df)
valid_dataloader = DataLoader(valid_dataset, shuffle=True)

valid_features, valid_labels = next(iter(valid_dataloader))

valid_features.shape, valid_labels.shape

(torch.Size([1, 6, 94]), torch.Size([1]))

# **LSTM**

In [ ]:
from torch import nn
import torch.nn.functional as F

class FallDetector(nn.Module):
    def __init__(self):
        super(FallDetector, self).__init__()
        self.lstm = nn.LSTM(6, 10, num_layers=1)
        self.hidden2tag = nn.Linear(10, 1)

    def forward(self, seq):
        output, (h_n, c_n) = self.lstm(seq.view(len(seq), 1, -1))
        tag_space = self.hidden2tag(c_n.view(1, -1))
        # print(tag_space.data)
        tag_scores = torch.sigmoid(tag_space)
        return tag_scores.view(-1)

**GRU (Gated Recurrent Units)**


In [ ]:
model = FallDetector()

In [ ]:
import torch.optim as optim
model = FallDetector()
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters()) # SGD & RMSprop & Adam : alternatives

In [ ]:
import numpy as np

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

# helper function, for calculating accuracy
def get_statistics(y_true, y_pred):
    y_pred = np.concatenate(tuple(y_pred)) > 0.5
    y_true = np.concatenate(tuple([[t for t in y] for y in y_true])).reshape(y_pred.shape)
    # print("Accuracy score: ", (y_true == y_pred).sum() / float(len(y_true)))
    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Recall: ", recall_score(y_true, y_pred))
    print(len(y_true + y_pred))
    print("Confusion Matrix")
    print(confusion_matrix(y_true, y_pred))

In [ ]:

for epoch in range(50): 
    labels = []
    preds = []
    loss = []

    i = 0
    for seq, label in train_dataloader:
        model.zero_grad()

        seq = seq.reshape((-1, 6))
        tag_scores = model(seq.to(torch.float))
        preds.append(tag_scores.data)
        labels.append(label.to(torch.float).data)

        loss = loss_function(tag_scores, label.to(torch.float))
        loss.backward()
        optimizer.step()


        i += 1
    print("Epoch Done")
    get_statistics(labels, preds)
  

Epoch Done
Accuracy:  0.6907216494845361
Recall:  1.0
97
Confusion Matrix
[[16 30]
 [ 0 51]]
Epoch Done
Accuracy:  0.7422680412371134
Recall:  0.9411764705882353
97
Confusion Matrix
[[24 22]
 [ 3 48]]
Epoch Done
Accuracy:  0.7938144329896907
Recall:  0.8823529411764706
97
Confusion Matrix
[[32 14]
 [ 6 45]]
Epoch Done
Accuracy:  0.8041237113402062
Recall:  0.8627450980392157
97
Confusion Matrix
[[34 12]
 [ 7 44]]
Epoch Done
Accuracy:  0.8144329896907216
Recall:  0.803921568627451
97
Confusion Matrix
[[38  8]
 [10 41]]
Epoch Done
Accuracy:  0.8556701030927835
Recall:  0.8823529411764706
97
Confusion Matrix
[[38  8]
 [ 6 45]]
Epoch Done
Accuracy:  0.8144329896907216
Recall:  0.8431372549019608
97
Confusion Matrix
[[36 10]
 [ 8 43]]
Epoch Done
Accuracy:  0.7525773195876289
Recall:  0.6666666666666666
97
Confusion Matrix
[[39  7]
 [17 34]]
Epoch Done
Accuracy:  0.8144329896907216
Recall:  0.9215686274509803
97
Confusion Matrix
[[32 14]
 [ 4 47]]
Epoch Done
Accuracy:  0.8247422680412371
Rec

In [ ]:

labels = []
preds = []
loss = []
model.eval()

for seq, label in valid_dataloader:

    seq = seq.reshape((-1, 6))
    tag_scores = model(seq.to(torch.float))
    preds.append(tag_scores.data)
    labels.append(label.to(torch.float).data)

print("Epoch Done")
get_statistics(labels, preds)

# label confusion matrix
# seaborn for making confusion matrix

Epoch Done
Accuracy:  0.8095238095238095
Recall:  0.75
21
Confusion Matrix
[[11  2]
 [ 2  6]]
